<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->
# <font color='blue'>Data Science Academy</font>
## <font color='blue'>IA Generativa e LLMs Para Processamento de Linguagem Natural</font>
## <font color='blue'>Estudo de Caso 2</font>
## <font color='blue'>Criando Aplicações Inteligentes com LangChain e LLMs</font>

## Instalando e Carregando Pacotes

In [1]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# !pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark.
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

O procedimento de instalação dos pacotes e dependências está no arquivo LEIAME.txt

In [2]:
# Langchain
#!pip install -q langchain

In [3]:
# Langchain para conectar no LLM da OpenAI
#!pip install -q langchain-openai

In [4]:
# VectorDB
#!pip install -q chromadb

In [5]:
# Dependência para o Langchain
#!pip install -q sqlalchemy

In [6]:
# Define o USER_AGENT
import os
os.environ["USER_AGENT"] = "Estudo de Caso 2"

In [7]:
# Imports
import os
import textwrap
import chromadb
import langchain
import sqlalchemy
import langchain_openai
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SimpleSequentialChain
from langchain.chains import SequentialChain
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferWindowMemory
from langchain.document_loaders import WebBaseLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
import warnings
warnings.filterwarnings('ignore')

In [8]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy"

Author: Data Science Academy



## Conhecendo o LangChain

Veja o e-book no Capítulo 12. Visite também a documentação oficial:

https://python.langchain.com/docs/get_started/introduction

## Definindo o LLM de Trabalho

In [9]:
# Coloque aqui sua API da OpenAI
os.environ['OPENAI_API_KEY'] = "coloque-aqui-sua-api"

In [10]:
# Define o LLM
# Cria uma instância de um Large Language Model (LLM), especificamente um fornecido pela OpenAI
dsa_llm = OpenAI(temperature = 0.9)

A temperatura é um hiperparâmetro que influencia a aleatoriedade das respostas geradas pelo modelo. Um valor mais alto de temperatura (geralmente variando entre 0 e 1) promove respostas mais criativas e variadas. Por outro lado, uma temperatura baixa tende a fazer com que o modelo produza respostas mais determinísticas e possivelmente mais previsíveis.

In [11]:
# Envia o prompt para o LLM e captura a resposta
nome = dsa_llm.invoke("Quero abrir um restaurante de comida japonesa. Sugira um nome chique para isso.")

Neste contexto, a string "Quero abrir um restaurante de comida japonesa. Sugira um nome chique para isso." serve como a prompt ou entrada para o modelo de linguagem. Ela descreve a tarefa que o usuário deseja que o modelo execute: a geração criativa de um nome para um novo restaurante de comida japonesa. O modelo utilizará seu treinamento em linguagem natural e conhecimento prévio para gerar uma resposta que atenda a essa solicitação.

In [12]:
print(nome)



"Kazoku Sushi & Dining" 


## Usando Prompt Templates

Os modelos de prompt (Prompt Templates) no contexto do LangChain referem-se a formas estruturadas de formatação da entrada para grandes modelos de linguagem (LLMs) para melhorar seu desempenho e adesão aos comportamentos desejados.

Um modelo de prompt define uma sequência de modelo com variáveis de espaço reservado que podem ser preenchidas dinamicamente. Isso permite que você construa prompts de maneira consistente e programática, em vez de codificar prompts completos.

Os modelos de prompt no LangChain fornecem uma maneira estruturada e extensível de interface com LLMs, facilitando a exploração e otimização de estratégias de prompt para melhorar o desempenho do modelo de linguagem em tarefas ou domínios específicos.

In [13]:
# Define o prompt template
dsa_prompt_template_name = PromptTemplate(
    input_variables = ['culinaria'],
    template = "Quero abrir um restaurante de comida {culinaria}. Sugira um nome chique para isso.")

A linha de código acima define um PromptTemplate, uma estrutura que permite a criação de prompts dinâmicos para serem utilizados com Large Language Models (LLMs). Essa abordagem é particularmente útil quando se deseja gerar prompts personalizados baseados em variáveis específicas ou quando se pretende reutilizar um formato de prompt com diferentes conjuntos de dados. 

**input_variables = ['culinaria']**: Define uma lista de variáveis que podem ser usadas para preencher o template. Neste caso, há uma única variável chamada 'culinaria'. Essa variável atua como um placeholder que será substituído por um valor específico quando o template for utilizado.

In [14]:
# Utiliza o template definido anteriormente para gerar um prompt específico, 
# inserindo o valor "Italiana" no lugar da variável culinaria
p = dsa_prompt_template_name.format(culinaria = "Italiana")

In [15]:
print(p)

Quero abrir um restaurante de comida Italiana. Sugira um nome chique para isso.


## Sequências de Operações com LLMChain

As chains no LangChain são sequências de operações que podem processar entradas e gerar saídas combinando vários componentes, incluindo grandes modelos de linguagem (LLMs), outras cadeias e ferramentas ou utilitários especializados.

Um LLMChain é um tipo de cadeia que permite interagir com um grande modelo de linguagem (LLM) de forma estruturada. Ele fornece uma interface simples para passar entradas para o LLM e recuperar suas saídas.

O LLMChain serve como um bloco de construção para muitas outras construções no LangChain, como agentes, ferramentas e tipos de cadeia mais avançados. Ao encapsular a lógica de interação do LLM em um componente reutilizável e extensível, o LLMChain simplifica o processo de construção de aplicativos que aproveitam grandes modelos de linguagem.

In [16]:
# Cria a chain
chain = LLMChain(llm = dsa_llm, prompt = dsa_prompt_template_name)

A linha de código acima cria uma instância de LLMChain, uma classe destinada a encadear ou sequenciar operações utilizando um LLM. Essa instância é configurada para usar um modelo de linguagem específico e um template de prompt predefinido. 

In [17]:
# Invoca a chain passando um parâmetro para o prompt
chain.invoke("Mexicana")

{'culinaria': 'Mexicana', 'text': '\n\n"El Sabroso Cantina"'}

In [18]:
# Cria a chain e ativa o verbose
chain = LLMChain(llm = dsa_llm, prompt = dsa_prompt_template_name, verbose = True)

In [19]:
# Invoca a chain passando um parâmetro para o prompt
chain.invoke("Mexicana")



> Entering new LLMChain chain...
Prompt after formatting:
Quero abrir um restaurante de comida Mexicana. Sugira um nome chique para isso.

> Finished chain.


{'culinaria': 'Mexicana', 'text': '\n\n"El Sabroso Cantina"'}

## Simple Sequential Chain

Um SimpleSequentialChain no LangChain é um tipo de cadeia que executa uma sequência de componentes (por exemplo, LLMs, ferramentas, outras cadeias) em uma ordem predefinida. É um dos tipos de cadeia mais básicos e comumente usados no LangChain.

Um exemplo de caso de uso de SimpleSequentialChain poderia ser um sistema de resposta a perguntas onde:

- O primeiro componente é um LLM que analisa a questão de entrada.
- O segundo componente é uma ferramenta que recupera documentos relevantes de um banco de dados.
- O terceiro componente é outro LLM que gera uma resposta com base na pergunta e nos documentos recuperados.

Ao encadear esses componentes em um SimpleSequentialChain, você pode criar um sistema mais complexo e capaz, mantendo uma arquitetura modular e extensível.

Embora SimpleSequentialChain seja útil para fluxos de trabalho lineares, LangChain também fornece outros tipos de cadeia como ConditionalChain e SequentialChain para fluxos de controle mais complexos e lógica de ramificação.

In [20]:
# Define o LLM com temperatura menor
dsa_llm = OpenAI(temperature = 0.6)

In [21]:
# Cria o prompt template
dsa_prompt_template_name = PromptTemplate(
    input_variables =['culinaria'],
    template = "Quero abrir um restaurante de comida {culinaria}. Sugira um nome chique para isso.")

In [22]:
# Cria a chain
dsa_chain_1 = LLMChain(llm = dsa_llm, prompt = dsa_prompt_template_name)

In [23]:
# Cria outro prompt template
dsa_prompt_template_items = PromptTemplate(
    input_variables = ['nome_restaurante'],
    template = """Sugira alguns itens do menu para {nome_restaurante}""")

In [24]:
# Cria a chain
dsa_chain_2 = LLMChain(llm = dsa_llm, prompt = dsa_prompt_template_items)

In [25]:
# Concatena as duas chains
dsa_chain_final = SimpleSequentialChain(chains = [dsa_chain_1, dsa_chain_2])

In [26]:
# Invoca a chain
dsa_chain_final.invoke("Indiana")

{'input': 'Indiana',
 'output': '\n\n1. Samosas (salgadinhos recheados com vegetais e especiarias)\n2. Chicken Tikka Masala (frango marinado em iogurte e cozido em molho cremoso de especiarias)\n3. Biryani (prato de arroz aromático com carne, frango ou vegetais)\n4. Naan (pão indiano tradicional assado na hora)\n5. Palak Paneer (queijo indiano em molho de espinafre e especiarias)\n6. Lamb Vindaloo (carne de cordeiro cozida lentamente em molho picante)\n7. Chana Masala (grão de bico cozido em molho de tomate e especiarias)\n8. Tandoori Chicken (frango marinado em iogurte e assado em forno de barro)\n9. Vegetable Korma (vegetais cozidos em molho de creme e especiarias)\n10. Mango Lassi (bebida refrescante de iogurte e manga).'}

## Sequential Chain

O SequentialChain é uma versão mais avançada do SimpleSequentialChain. Enquanto o SimpleSequentialChain executa uma sequência fixa de componentes, o SequentialChain permite a execução dinâmica e condicional de componentes com base nas saídas de componentes anteriores.

Um exemplo de caso de uso de SequentialChain poderia ser um agente de conversação que:

- Usa um LLM para entender a entrada do usuário e determinar a ação apropriada.
- Executa condicionalmente diferentes componentes (por exemplo, pesquisa de banco de dados, chamada de API, cálculo) com base na saída do LLM.
- Opcionalmente, solicita ao usuário informações adicionais, se necessário.
- Gera uma resposta final usando outro LLM, com base nas saídas dos componentes anteriores.

Ao aproveitar o SequentialChain, você pode criar aplicativos mais inteligentes e adaptáveis que podem ajustar dinamicamente seu comportamento com base em resultados e estados intermediários.

<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->

In [27]:
# Define o LLM
dsa_llm = OpenAI(temperature = 0.7)

In [28]:
# Criando a primeira chain

# Define o prompt template
dsa_prompt_template_name = PromptTemplate(
    input_variables = ['culinaria'],
    template = "Quero abrir um restaurante de comida {culinaria}. Sugira um nome chique para isso.")

# Define a chain com um parâmetro de saída
dsa_chain_1 = LLMChain(llm = dsa_llm, prompt = dsa_prompt_template_name, output_key = "nome_restaurante")

In [29]:
# Criando a segunda chain

# Define o prompt template
dsa_prompt_template_items = PromptTemplate(
    input_variables = ['nome_restaurante'],
    template = "Sugira alguns itens do menu para {nome_restaurante}."
)

# Define a chain com um parâmetro de saída
dsa_chain_2 = LLMChain(llm = dsa_llm, prompt = dsa_prompt_template_items, output_key = "itens_menu")

In [30]:
# Cria a sequência de chains
dsa_chain = SequentialChain(chains = [dsa_chain_1, dsa_chain_2],
                            input_variables = ['culinaria'],
                            output_variables = ['nome_restaurante', "itens_menu"])

In [31]:
dsa_chain.invoke({"culinaria": "Italiana"})

{'culinaria': 'Italiana',
 'nome_restaurante': '\n\n"La Bella Cucina"',
 'itens_menu': '\n\n1. Bruschetta de tomate e manjericão\n2. Salada Caprese\n3. Lasanha à bolonhesa\n4. Fettuccine Alfredo\n5. Risoto de cogumelos\n6. Frango Parmigiana\n7. Bife à Fiorentina\n8. Pizza Margherita\n9. Cannelloni de espinafre e ricota\n10. Tiramisu.'}

In [32]:
# Invocação do método e captura da resposta
resposta = dsa_chain.invoke({"culinaria": "Italiana"})

# Preparando a string formatada
saida_formatada = f"Culinária: {resposta['culinaria']}\nNome do Restaurante: {resposta['nome_restaurante'].strip()}\nItens do Menu:"

# Adicionando cada item do menu à string formatada
itens_menu = resposta['itens_menu'].strip().split('\n')
for item in itens_menu:
    saida_formatada += f"\n{item}"

# Exibindo a saída formatada
print(saida_formatada)

Culinária: Italiana
Nome do Restaurante: "Dolce Vita Ristorante"
Itens do Menu:
1. Bruschettas variadas (tomate e manjericão, presunto de Parma e queijo brie, cogumelos trufados)
2. Lasanha à bolonhesa
3. Risoto de camarão com limão siciliano
4. Filé mignon ao molho gorgonzola
5. Gnocchi de batata ao molho pesto
6. Ravióli de ricota com espinafre ao molho tomate fresco
7. Salada caprese (tomate, muçarela de búfala, manjericão)
8. Salmão grelhado com crosta de ervas e risoto de alho poró
9. Panna cotta com calda de frutas vermelhas
10. Tiramisu tradicional italiano.


## Criando Memória Para o LLM

No LangChain, "Memória" refere-se a componentes que permitem que cadeias, agentes e outras construções armazenem e recuperem informações de entradas, saídas e estados intermediários anteriores. Isso lhes permite manter o contexto e fazer uso de informações relevantes do histórico de conversas ou de cálculos anteriores.

<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->

In [33]:
# Define a chain
dsa_chain = LLMChain(llm = dsa_llm, prompt = dsa_prompt_template_name)

In [34]:
# Invoca a chain
nome = dsa_chain.invoke("Mexicana")
print(nome)

{'culinaria': 'Mexicana', 'text': '\n\n"Casa Mexicana"\n"El Sabor Auténtico"\n"Sabores del Sur"\n"La Frontera"\n"Chilaquiles y Más"\n"El Ranchito"\n"Los Sabores de México"\n"La Cantina Mexicana"\n"El Patio Mexicano"\n"La Hacienda de los Sabores"\n"El Gran Taco"\n"El Sabor de México"\n"Cocina Mexicana Moderna"\n"La Fiesta Mexicana"\n"El Árbol de México"'}


In [35]:
# Invoca a chain
nome = dsa_chain.invoke("Argentina")
print(nome)

{'culinaria': 'Argentina', 'text': '\n\n"La Parrilla de Buenos Aires" '}


In [36]:
dsa_chain.memory

In [37]:
type(dsa_chain.memory)

NoneType

In [38]:
# Criando o objeto de memória
memory = ConversationBufferMemory()

In [39]:
dsa_chain = LLMChain(llm = dsa_llm, prompt = dsa_prompt_template_name, memory = memory)

In [40]:
nome = dsa_chain.run("Mexicana")
print(nome)



"Cantina del Sol"


In [41]:
nome = dsa_chain.run("Argentina")
print(nome)



"Tango Gastronômico"


In [42]:
print(dsa_chain.memory.buffer)

Human: Mexicana
AI: 

"Cantina del Sol"
Human: Argentina
AI: 

"Tango Gastronômico"


## Conversation Chain

Um ConversationChain é um tipo especializado de chain projetado para lidar com conversas ou diálogos multivoltas com um LLM.

O ConversationChain é particularmente útil para criar agentes de conversação, chatbots ou qualquer aplicativo que exija a manutenção do contexto durante vários turnos de interação com um usuário. Ao abstrair as complexidades do gerenciamento do histórico de conversas e da formatação de prompts, o ConversationChain simplifica o processo de construção de sistemas de diálogo multiturno com LLMs.

In [43]:
# Cria o objeto de conversação
conv = ConversationChain(llm = OpenAI(temperature = 0.7))

In [44]:
print(conv.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [45]:
conv.invoke("Que país venceu mais vezes a Copa do Mundo de Futebol?")

{'input': 'Que país venceu mais vezes a Copa do Mundo de Futebol?',
 'history': '',
 'response': ' O Brasil venceu a Copa do Mundo de Futebol cinco vezes, nos anos de 1958, 1962, 1970, 1994 e 2002. A Itália e a Alemanha venceram quatro vezes cada, enquanto a Argentina e o Uruguai venceram duas vezes cada. A França, a Inglaterra e a Espanha venceram uma vez cada. E apenas oito países venceram a Copa do Mundo até agora.'}

In [46]:
conv.invoke("Quanto é 30 + 12?")

{'input': 'Quanto é 30 + 12?',
 'history': 'Human: Que país venceu mais vezes a Copa do Mundo de Futebol?\nAI:  O Brasil venceu a Copa do Mundo de Futebol cinco vezes, nos anos de 1958, 1962, 1970, 1994 e 2002. A Itália e a Alemanha venceram quatro vezes cada, enquanto a Argentina e o Uruguai venceram duas vezes cada. A França, a Inglaterra e a Espanha venceram uma vez cada. E apenas oito países venceram a Copa do Mundo até agora.',
 'response': '  O resultado de 30 + 12 é 42.'}

In [47]:
conv.invoke("Quem é o maior artilheiro da história das Copas do Mundo de Futebol?")

{'input': 'Quem é o maior artilheiro da história das Copas do Mundo de Futebol?',
 'history': 'Human: Que país venceu mais vezes a Copa do Mundo de Futebol?\nAI:  O Brasil venceu a Copa do Mundo de Futebol cinco vezes, nos anos de 1958, 1962, 1970, 1994 e 2002. A Itália e a Alemanha venceram quatro vezes cada, enquanto a Argentina e o Uruguai venceram duas vezes cada. A França, a Inglaterra e a Espanha venceram uma vez cada. E apenas oito países venceram a Copa do Mundo até agora.\nHuman: Quanto é 30 + 12?\nAI:   O resultado de 30 + 12 é 42.',
 'response': ' O maior artilheiro da história das Copas do Mundo de Futebol é o jogador alemão Miroslav Klose, com 16 gols marcados em quatro Copas do Mundo. Ele superou o recorde anterior de 15 gols, que pertencia ao brasileiro Ronaldo.'}

In [48]:
print(conv.memory.buffer)

Human: Que país venceu mais vezes a Copa do Mundo de Futebol?
AI:  O Brasil venceu a Copa do Mundo de Futebol cinco vezes, nos anos de 1958, 1962, 1970, 1994 e 2002. A Itália e a Alemanha venceram quatro vezes cada, enquanto a Argentina e o Uruguai venceram duas vezes cada. A França, a Inglaterra e a Espanha venceram uma vez cada. E apenas oito países venceram a Copa do Mundo até agora.
Human: Quanto é 30 + 12?
AI:   O resultado de 30 + 12 é 42.
Human: Quem é o maior artilheiro da história das Copas do Mundo de Futebol?
AI:  O maior artilheiro da história das Copas do Mundo de Futebol é o jogador alemão Miroslav Klose, com 16 gols marcados em quatro Copas do Mundo. Ele superou o recorde anterior de 15 gols, que pertencia ao brasileiro Ronaldo.


## Conversation Buffer Window Memory

ConversationBufferWindowMemory é um tipo de componente de memória do LangChain projetado especificamente para uso com cadeias de conversação (ConversationChain). Ele fornece uma maneira de armazenar e recuperar o histórico de conversas enquanto limita a quantidade de contexto retido com base em um tamanho de janela especificado.

A principal vantagem do ConversationBufferWindowMemory é sua capacidade de limitar a quantidade de contexto fornecido ao LLM, o que pode ser importante para o desempenho e evitar que o modelo fique sobrecarregado com muitas informações irrelevantes. Ao ajustar o tamanho da janela, você pode controlar a compensação entre fornecer contexto suficiente e evitar sobrecarga computacional excessiva.

Esse tipo de memória é particularmente útil para criar agentes de conversação, chatbots ou qualquer aplicativo que exija a manutenção de uma janela contínua relevante do histórico de conversas recentes para contexto.

<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->

In [49]:
# Define a janela de memória
memory = ConversationBufferWindowMemory(k = 1)

In [50]:
# Cria a chain de conversação
conv = ConversationChain(llm = OpenAI(temperature = 0.7), memory = memory)

In [51]:
# Invoca o LLM
conv.run("Quem venceu a primeira Copa do Mundo de Futebol?")

' A primeira Copa do Mundo de Futebol foi vencida pelo Uruguai, em 1930. Eles derrotaram a Argentina por 4 a 2 na final, que foi disputada em Montevidéu. O primeiro gol da história da Copa do Mundo foi marcado pelo jogador francês Lucien Laurent, na partida entre França e México. No total, 13 equipes participaram do torneio, com a presença de 32 jogadores.'

In [52]:
# Invoca o LLM
conv.invoke("Quanto é 10 + 19?")

{'input': 'Quanto é 10 + 19?',
 'history': 'Human: Quem venceu a primeira Copa do Mundo de Futebol?\nAI:  A primeira Copa do Mundo de Futebol foi vencida pelo Uruguai, em 1930. Eles derrotaram a Argentina por 4 a 2 na final, que foi disputada em Montevidéu. O primeiro gol da história da Copa do Mundo foi marcado pelo jogador francês Lucien Laurent, na partida entre França e México. No total, 13 equipes participaram do torneio, com a presença de 32 jogadores.',
 'response': '  10 + 19 é igual a 29.'}

In [53]:
# Invoca o LLM
conv.invoke("Quem era o capitão da seleção campeã da primeira Copa do Mundo de Futebol?")

{'input': 'Quem era o capitão da seleção campeã da primeira Copa do Mundo de Futebol?',
 'history': 'Human: Quanto é 10 + 19?\nAI:   10 + 19 é igual a 29.',
 'response': ' O capitão da seleção campeã da primeira Copa do Mundo de Futebol foi o uruguaio José Nasazzi. Ele liderou a equipe em 1930, quando o Uruguai conquistou o título em casa. Ele também foi um dos melhores defensores do torneio, ajudando sua equipe a manter o título invicto.'}

In [54]:
print(conv.memory.buffer)

Human: Quem era o capitão da seleção campeã da primeira Copa do Mundo de Futebol?
AI:  O capitão da seleção campeã da primeira Copa do Mundo de Futebol foi o uruguaio José Nasazzi. Ele liderou a equipe em 1930, quando o Uruguai conquistou o título em casa. Ele também foi um dos melhores defensores do torneio, ajudando sua equipe a manter o título invicto.


## LangChain e VectorDB com Dados Extraídos via Web Scraping

ChromaDB é uma biblioteca de banco de dados de armazenamento vetorial que se integra ao LangChain. Ele fornece funcionalidade para armazenar, recuperar e pesquisar grandes quantidades de dados de texto de maneira eficiente, usando incorporações de vetores (embeddings) e similaridade semântica.

https://www.trychroma.com/

https://pypi.org/project/chromadb/

In [55]:
# Extração de dados da web
dsa_dados = WebBaseLoader(
    "https://blog.dsacademy.com.br/como-rag-retrieval-augmented-generation-funciona-para-personalizar-os-llms/"
)

Nota: Sempre verifique o robots.txt de um web site antes de fazer raspagem de dados. Não faça raspagem se não for permitido!

In [56]:
# Carrega os documentos
documentos = dsa_dados.load()

In [57]:
len(documentos)

1

In [58]:
# Extrai o primeiro documento (neste caso há um único documento)
document = documentos[0]

In [59]:
# Chaves do dicionário
document.__dict__.keys()

dict_keys(['id', 'metadata', 'page_content', 'type'])

In [60]:
# Visualiza os 100 primeiros caracteres
document.page_content[:100]

'\n\n\n\n\n\n \n\n\n\n\nComo RAG (Retrieval-Augmented Generation) Funciona Para Personalizar os LLMs? - Data Sci'

In [61]:
# Metadados
document.metadata

{'source': 'https://blog.dsacademy.com.br/como-rag-retrieval-augmented-generation-funciona-para-personalizar-os-llms/',
 'title': 'Como RAG (Retrieval-Augmented Generation) Funciona Para Personalizar os LLMs? - Data Science Academy',
 'description': 'Com o uso de RAG, é introduzido um componente de recuperação de informações que utiliza a entrada do usuário para extrair informações de uma nova fonte de dados primeiro. A consulta do usuário e as informações relevantes são fornecidas ao LLM. O LLM usa esse novo conhecimento e seus dados de treinamento para criar respostas mais adaptadas.',
 'language': 'pt-BR'}

In [62]:
# Cria o vector store (ajustado para versão mais recente do LangChain)
embedding = OpenAIEmbeddings() 
index_creator = VectorstoreIndexCreator(embedding = embedding)
index = index_creator.from_loaders([dsa_dados])
#index = VectorstoreIndexCreator().from_loaders([dsa_dados])

A linha de código acima envolve a criação de um índice para um armazenamento de vetores, comumente usado em tarefas de busca e recuperação de informações baseadas em similaridade de conteúdo. Essa operação é essencial em sistemas que empregam técnicas de inteligência artificial e aprendizado de máquina para organizar e recuperar dados de maneira eficiente. 

Enquanto um "Vector Store" é um conceito mais genérico relacionado ao armazenamento de vetores, um "Vector Database" é um sistema de banco de dados especializado projetado para lidar com vetores de alta dimensão e otimizado para consultas de similaridade e outras operações relacionadas a vetores. Usaremos o ChromaDB para criar um banco de dados vetorial.

In [63]:
# Função para imprimir resultado formatado
def dsa_print_response(response: str):
    print("\n".join(textwrap.wrap(response, width = 100)))

In [64]:
# Define uma query
query = """
Você é um Engenheiro de IA de Nível Sênior.
Explique o funcionamento de RAG na construção de aplicações inteligentes.
"""

In [65]:
# Imprime a resposta ao consultar o índice (ajustado para versão mais recente do LangChain)
response = index.query(query, llm = OpenAI(temperature = 0.7))
dsa_print_response(response)

 RAG (Retrieval-Augmented Generation) é uma técnica que combina o uso de grandes modelos de
linguagem (LLMs) com a recuperação de informações a partir de fontes de conhecimento confiáveis.
Isso permite que o LLM obtenha informações relevantes e confiáveis antes de gerar uma resposta,
aumentando a precisão e confiabilidade das aplicações inteligentes. O processo envolve a criação de
dados externos a partir de fontes diversas, como APIs e bancos de dados, e a utilização desses dados
em conjunto com os dados de treinamento do LLM para gerar respostas mais personalizadas e adaptadas
às necessidades dos usuários.


> Vamos agora trabalhar com um Vector DB.

In [66]:
# Cria um template
dsa_template = """Você é um Engenheiro de IA de Nível Sênior.

{context}

Responda considerando as técnicas mais modernas que você conhecer.

Peegunta: {question}
Resposta:"""

In [67]:
# Cria o prompt
prompt = PromptTemplate(template = dsa_template, input_variables = ["context", "question"])

In [68]:
# Imprime o prompt para visualizar o formato
print(
    prompt.format(
        context = "Aplicação de IA para sistemas de atendimento ao cliente.",
        question = "Como criar uma aplicação web com LLM?",
    )
)

Você é um Engenheiro de IA de Nível Sênior.

Aplicação de IA para sistemas de atendimento ao cliente.

Responda considerando as técnicas mais modernas que você conhecer.

Peegunta: Como criar uma aplicação web com LLM?
Resposta:


In [69]:
# Cria o objeto de embeddings
embeddings = OpenAIEmbeddings()

A linha de código acima refere-se à inicialização de uma instância da classe OpenAIEmbeddings, que é uma interface para gerar embeddings (representações vetoriais) usando os modelos fornecidos pela OpenAI, como os modelos de linguagem GPT. Os embeddings são transformações de dados brutos, como textos, em vetores de números fixos, capturando aspectos semânticos e contextuais do conteúdo original de maneira que possa ser processado por algoritmos de aprendizado de máquina.

In [70]:
# Cria o VectorDB convertendo os documentos de texto em representações numéricas (embeddings)
dsa_db = Chroma.from_documents(documentos, embeddings)

A linha de código acima é para criação de um banco de dados vetorial utilizando Chroma. Essa operação envolve a preparação de uma estrutura de dados otimizada para buscas e análises baseadas em documentos e seus respectivos embeddings. 

In [71]:
type(dsa_db)

langchain_community.vectorstores.chroma.Chroma

In [72]:
# Argumentos
chain_type_kwargs = {"prompt": prompt}

In [73]:
# Chain de RetrievalQA
chain = RetrievalQA.from_chain_type(llm = ChatOpenAI(temperature = 0),
                                    chain_type = "stuff",
                                    retriever = dsa_db.as_retriever(search_kwargs = {"k": 1}),
                                    chain_type_kwargs = chain_type_kwargs)

A linha de código acima está criando uma instância chamada chain, utilizando a classe RetrievalQA para configurar uma cadeia de processos focada em realizar tarefas de Question Answering (QA) com base em recuperação de informações. O método from_chain_type é utilizado para especificar o tipo de cadeia de processos e configurar seus componentes principais, como o modelo de linguagem e o mecanismo de recuperação. 

In [74]:
# Consulta
query = "Explique o que é RAG em 5 sentenças"

In [75]:
# Resposta
resposta = chain.invoke(query)

In [76]:
resposta

{'query': 'Explique o que é RAG em 5 sentenças',
 'result': 'RAG, ou Retrieval-Augmented Generation, é uma técnica que combina a geração de linguagem por modelos de Machine Learning com a recuperação de informações relevantes de fontes externas. Essa abordagem permite que o modelo de linguagem seja enriquecido com dados adicionais, tornando suas respostas mais contextualizadas e precisas. O processo envolve a criação de dados externos, a recuperação de informações relevantes, o enriquecimento dos prompts fornecidos ao modelo e a atualização contínua dos dados externos. Ao utilizar o RAG, as organizações podem melhorar a qualidade e confiabilidade das respostas geradas pelo modelo de linguagem.'}

## Criando Chatbot Especialista em Vendas com LangChain e LLM

In [77]:
# Definindo o LLM
dsa_gpt = ChatOpenAI(temperature = 0)

In [78]:
# Template
template = """Esta é uma conversa entre um cliente e um especialista em venda de carros esportivos. 
                Você é o especialista em carros, conhece bem os modelos esportivos e deve sempre responder 
                com a maior precisão possível.

Current conversation:
{history}
Human: {input}
CarSpecialist:"""

In [79]:
# Cria o prompt template
dsa_prompt = PromptTemplate(input_variables = ["history", "input"], template = template)

In [80]:
# Cria a chain de conversação
conversation = ConversationChain(prompt = dsa_prompt,
                                 llm = dsa_gpt,
                                 verbose = False,
                                 memory = ConversationBufferMemory(ai_prefix = "CarSpecialist"))

In [81]:
# Loop de conversação limitado a 5 interações (aumente o número de interações ou remova o bloco if)

# Inicializa o contador
contador = 0  

# Loop
while True:
    
    prompt = input(prompt = "Cliente: ")
    print()
    resultado = conversation(prompt)
    dsa_print_response("Especialista: " + resultado["response"])
    print()
    
    contador += 1  
    
    if contador >= 5:  
        print('\nObrigado Por Usar o Sistema de Atendimento Baseado em IA!')
        break  

Cliente:  Olá. Tudo bem?



Especialista: Olá! Tudo bem e você? Como posso te ajudar hoje?



Cliente:  Qual o modelo mais novo de Ferrari?



Especialista:  O modelo mais novo da Ferrari é o Ferrari SF90 Stradale, lançado em 2019. É um
supercarro híbrido plug-in com um motor V8 biturbo e três motores elétricos, totalizando uma
potência de mais de 1.000 cavalos. É um verdadeiro monstro em termos de desempenho e tecnologia.



Cliente:  Qual o torque?



Especialista:  O Ferrari SF90 Stradale possui um torque impressionante de 800 Nm. Isso significa que
ele tem uma incrível capacidade de aceleração e força nas rodas. É um carro realmente potente e
emocionante de dirigir.



Cliente:  Qual a garantia do motor?



Especialista:  A garantia do motor do Ferrari SF90 Stradale é de 7 anos, o que demonstra a confiança
da Ferrari na qualidade e durabilidade do motor deste supercarro. Além disso, a Ferrari oferece um
serviço de assistência técnica especializada para garantir o bom funcionamento do motor ao longo do
tempo.



Cliente:  Como compro a Ferrari?



Especialista: Para comprar um Ferrari SF90 Stradale, você pode entrar em contato com uma
concessionária autorizada da Ferrari ou diretamente com a marca. Eles irão te fornecer todas as
informações necessárias sobre o processo de compra, formas de pagamento e entrega do veículo. Além
disso, é importante verificar a disponibilidade do modelo e possíveis opções de personalização de
acordo com suas preferências.


Obrigado Por Usar o Sistema de Atendimento Baseado em IA!


In [82]:
%reload_ext watermark
%watermark -a "Data Science Academy"

Author: Data Science Academy



In [83]:
#%watermark -v -m

In [84]:
#%watermark --iversions

# Fim